In [34]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import math
import osmnx as ox
                
G = ox.graph_from_place('Banepa, Nepal', network_type='drive')

def find_nearest_node(G, point):
    return min(G.nodes, key=lambda x: math.dist((G.nodes[x]['y'], G.nodes[x]['x']), point))

class RouteOptimizer:

    def __init__(self, G, landmark_location, destination_location):
        self.G = G
        self.landmark_location = find_nearest_node(G, landmark_location)
        self.destination_location = find_nearest_node(G, destination_location)
        
    def get_shortest_path(self):
        # impute missing edge speed and add travel times
        self.G = ox.add_edge_speeds(self.G)
        self.G = ox.add_edge_travel_times(self.G)

        # calculate shortest path minimizing travel time
        orig = self.landmark_location
        dest = self.destination_location
        self.path = nx.shortest_path(self.G, orig, dest, 'travel_time')
        return self.path
    
    
    def calculate_initial_compass_bearing(self, pointA, pointB):
        lat1 = math.radians(pointA[0])
        lat2 = math.radians(pointB[0])
        diffLong = math.radians(pointB[1] - pointA[1])
        x = math.sin(diffLong) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))
        initial_bearing = math.atan2(x, y)
        initial_bearing = math.degrees(initial_bearing)
        compass_bearing = (initial_bearing + 360) % 360
        return compass_bearing

from math import radians, sin, cos, sqrt, atan2

def haversine_distance(coord1, coord2):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = radians(lat1), radians(lat2) 
    dphi       = radians(lat2 - lat1)
    dlambda    = radians(lon2 - lon1)
    
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlambda/2)**2
    return 2*R*atan2(sqrt(a), sqrt(1 - a))

# Hash Generation Function
def generate_hash(landmark_name, path, optimizer):
    directions = []
    prev_dir = None
    sum_length = 0

    for i in range(len(path) - 1):
        # Retrieve the coordinates of the nodes from the graph
        pointA = (optimizer.G.nodes[path[i]]['y'], optimizer.G.nodes[path[i]]['x'])
        pointB = (optimizer.G.nodes[path[i + 1]]['y'], optimizer.G.nodes[path[i + 1]]['x'])
        
        bearing = optimizer.calculate_initial_compass_bearing(pointA, pointB)
        length = haversine_distance(pointA, pointB)
        
        if (bearing >= 0 and bearing < 45) or (bearing >= 315 and bearing < 360):
            dir = "N"
        elif bearing >= 45 and bearing < 135:
            dir = "E"
        elif bearing >= 135 and bearing < 225:
            dir = "S"
        else:
            dir = "W"

        if prev_dir == dir:
            sum_length += length
        else:
            if prev_dir is not None:
                directions.append(f"{prev_dir}_{int(sum_length)}")
            sum_length = length
            prev_dir = dir

    # Add the last direction and length
    if prev_dir is not None:
        directions.append(f"{prev_dir}_{int(sum_length)}")

    hash_string = f"{landmark_name}|{'|'.join(directions)}"
    return hash_string
    

In [35]:
# Example usage:
destination_location = (27.632028, 85.504183)
landmark_location = [27.632278, 85.501588]


optimizer = RouteOptimizer(G, landmark_location, destination_location)
path = optimizer.get_shortest_path()
# print(path)
hash_string = generate_hash('Landmark1', path, optimizer)
print(f"For building at (30.702, 90.305), hash_string is: {hash_string}")

For building at (30.702, 90.305), hash_string is: Landmark1|E_248


In [36]:
# create folium web map
route_map = ox.plot_route_folium(G, path)
route_map

/var/folders/zz/qg9mqbp56631kxmjjyl11f1h0000gn/T/ipykernel_18647/3834381230.py:2: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  route_map = ox.plot_route_folium(G, path)
